# Aggregation functions

Aggregation functions operate on arrays of data and produce a single numerical result.

The next cell starts the container for the playground database.

In [2]:
docker run --rm -d\
    -e POSTGRES_PASSWORD=docker_app \
    --name aggregation_postgres_exmaples \
    postgres:15.4 &> /dev/null
sleep 5

**Note** Don't forget to stop the container when you've finished playing with it.

In [1]:
docker stop aggregation_postgres_exmaples;

aggregation_postgres_exmaples


## Logical operations (BOOL_OR, BOOL_AND)

Sometimes you need to find out if all values of a boolean variable by group are true. For such cases you can use `boot_and` aggregation function. Also there is function `bool_or` if you need to get if any value by group is true.

---

The following example presents a comprehensive list of all possible pairings between logical values and the functions we're exploting are applied to them. 

In [11]:
docker exec -i aggregation_postgres_exmaples psql -U postgres << EOF
SELECT
    bool_and(all_true) AS all_true_and,
    bool_or(all_true) AS all_true_or,
    bool_and(one_true1) AS one_true1_and,
    bool_or(one_true1) AS one_true1_or,
    bool_and(one_true2) AS one_true2_and,
    bool_or(one_true2) AS one_true2_or,
    bool_and(no_true) AS no_true_and,
    bool_and(no_true) AS no_true_or
FROM (
    VALUES 
        (TRUE, TRUE, FALSE, FALSE), 
        (TRUE, FALSE, TRUE, FALSE)
) AS temp(all_true, one_true1, one_true2, no_true);
EOF

 all_true_and | all_true_or | one_true1_and | one_true1_or | one_true2_and | one_true2_or | no_true_and | no_true_or 
--------------+-------------+---------------+--------------+---------------+--------------+-------------+------------
 t            | t           | f             | t            | f             | t            | f           | f
(1 row)



## Strings concatenations

If you need to combine the values of a string variable into a single string, you can use the `string_agg(<column for concatenation>, <separator>)` aggregation function.  This function will output all the values in a single line, separated by the specified separator. 

---

The following example concatenates strings 'one', 'two' and 'three'.

In [14]:
docker exec -i aggregation_postgres_exmaples psql -U postgres << EOF
SELECT 
    string_agg(string_variable, ', ')
FROM (
    VALUES
    ('one'), ('two'), ('three')
) AS temp(string_variable)
EOF

   string_agg    
-----------------
 one, two, three
(1 row)



## Collect array

There is a function that allows transforming a set of numbers into an array that fits in one cell of a table. In PostgreSQL, this can be done using the `array_agg` function.

---

The following cell shows the application of the `array_agg` function to the `some_value` column, excluding a particular element from the array.

In [2]:
docker exec -i sql_functions_example psql -U postgres -d postgres << EOF
SELECT 
    array_agg(some_value) whole_array,
    (array_agg(some_value))[1] first_element,
    (array_agg(some_value))[2] second_element,
    (array_agg(some_value))[3] third_element
FROM (
    VALUES
    (4), (3), (3), (1), (3), (3), (3), (5), (5), (1)
) AS temp(some_value);
EOF

      whole_array      | first_element | second_element | third_element 
-----------------------+---------------+----------------+---------------
 {4,3,3,1,3,3,3,5,5,1} |             4 |              3 |             3
(1 row)

